In [23]:
from tensorflow import keras
import tensorflow as tf

In [24]:
fout = f'models/cnn1'

In [25]:
from tensorflow.keras import layers

In [26]:
model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3)))
model.add(layers.Conv2D(32, 5, strides=2, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))

# Can you guess what the current output shape is at this point? Probably not.
# Let's just print it:
model.summary()

# The answer was: (40, 40, 32), so we can keep downsampling...

model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))

# And now?
model.summary()

# Now that we have 4x4 feature maps, time to apply global max pooling.
model.add(layers.GlobalMaxPooling2D())

# Finally, we add a classification layer.
model.add(layers.Dense(1))

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 123, 123, 32)      2432      
                                                                 
 conv2d_25 (Conv2D)          (None, 121, 121, 32)      9248      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 40, 40, 32)       0         
 g2D)                                                            
                                                                 
Total params: 11,680
Trainable params: 11,680
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 123, 123, 32)      2432      
                            

In [27]:
fdata = f'../../data/sars-cov2-ct-scan'
from pathlib import Path
p = Path(fdata)
ptrain = p / 'train'
ptest = p / 'test'
pval = p / 'val'

In [28]:
BATCH_SIZE = 10

w, h = (250, 250)
from keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(rescale=1. / 255)

val_generator = ImageDataGenerator(rescale=1. / 255)

test_generator = ImageDataGenerator(rescale=1. / 255)

In [29]:
traingen = train_generator.flow_from_directory(ptrain,
                                               target_size=(w, h),
                                               batch_size=BATCH_SIZE, 
                                               shuffle=True,
                                               seed=42)

validgen = val_generator.flow_from_directory(pval,
                                               target_size=(w, h),
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=42)

testgen = test_generator.flow_from_directory(ptest,
                                             target_size=(w, h),
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)

Found 1788 images belonging to 2 classes.
Found 198 images belonging to 2 classes.
Found 495 images belonging to 2 classes.


In [31]:
foldername = fout
save_model = tf.keras.callbacks.ModelCheckpoint(
    filepath=foldername+'/tl-cnn1_model_{epoch:02d}.h5')

save_weight = tf.keras.callbacks.ModelCheckpoint(
    filepath=foldername+'/tl-cnn1_weight_{epoch:02d}.h5',
    save_weights_only=True)

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=[keras.metrics.BinaryAccuracy()])
history = model.fit(traingen, epochs=20, validation_data= validgen,
             callbacks=[save_model, save_weight])

Epoch 1/20
179/179 [==============================] - 43s 233ms/step - loss: 7.6246 - binary_accuracy: 0.5000 - val_loss: 7.6246 - val_binary_accuracy: 0.5000
Epoch 2/20
179/179 [==============================] - 8s 45ms/step - loss: 7.6246 - binary_accuracy: 0.5000 - val_loss: 7.6246 - val_binary_accuracy: 0.5000
Epoch 3/20
179/179 [==============================] - 7s 37ms/step - loss: 7.6246 - binary_accuracy: 0.5000 - val_loss: 7.6246 - val_binary_accuracy: 0.5000
Epoch 4/20
179/179 [==============================] - 7s 37ms/step - loss: 7.6246 - binary_accuracy: 0.5000 - val_loss: 7.6246 - val_binary_accuracy: 0.5000
Epoch 5/20
179/179 [==============================] - 7s 37ms/step - loss: 7.6246 - binary_accuracy: 0.5000 - val_loss: 7.6246 - val_binary_accuracy: 0.5000
Epoch 6/20
179/179 [==============================] - 7s 36ms/step - loss: 7.6246 - binary_accuracy: 0.5000 - val_loss: 7.6246 - val_binary_accuracy: 0.5000
Epoch 7/20
179/179 [==============================] - 7s